In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use('Agg')

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [2]:
# ArgParser Things
dataset = "animals"
plot = "output/train_val.png"
modelPath = "output/train_simple_nn.model"
label_bin = "output/train_simple_nn_lb.pickle"

# Loading the dataset
<img src="https://pyimagesearch.com/wp-content/uploads/2018/09/keras_tutorial_step2.png">

In [3]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)
# loop over the input images
for imagePath in imagePaths:
	# load the image, resize the image to be 32x32 pixels (ignoring
	# aspect ratio), flatten the image into 32x32x3=3072 pixel image
	# into a list, and store the image in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (32, 32)).flatten()
	data.append(image)
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

[INFO] loading images...


In [4]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# Construct your training and testing splits
<img src="https://pyimagesearch.com/wp-content/uploads/2018/09/keras_tutorial_step3.png">

In [5]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.25, random_state=42)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# Define your Keras model architecture
<img src="https://www.pyimagesearch.com/wp-content/uploads/2018/09/keras_tutorial_simplenn_arch.png">

In [6]:
# define the 3072-1024-512-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

# Compile your Keras model
<img src="https://www.pyimagesearch.com/wp-content/uploads/2018/09/keras_tutorial_step5.png">

In [7]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 80

# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] training network...


# Fit your Keras model to the data
<img src="https://www.pyimagesearch.com/wp-content/uploads/2018/09/keras_tutorial_step6.png">

In [8]:
# train the nueral network
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=32)

Train on 2250 samples, validate on 750 samples
Epoch 1/80
2250/2250 [==============================] - 2s 977us/sample - loss: 1.1010 - accuracy: 0.3640 - val_loss: 1.0948 - val_accuracy: 0.3147
Epoch 2/80
2250/2250 [==============================] - 2s 762us/sample - loss: 1.0774 - accuracy: 0.4156 - val_loss: 1.0714 - val_accuracy: 0.3573
Epoch 3/80
2250/2250 [==============================] - 2s 764us/sample - loss: 1.0615 - accuracy: 0.4369 - val_loss: 1.0564 - val_accuracy: 0.3920
Epoch 4/80
2250/2250 [==============================] - 2s 804us/sample - loss: 1.0474 - accuracy: 0.4529 - val_loss: 1.0311 - val_accuracy: 0.5040
Epoch 5/80
2250/2250 [==============================] - 2s 741us/sample - loss: 1.0259 - accuracy: 0.4764 - val_loss: 1.0406 - val_accuracy: 0.4827
Epoch 6/80
2250/2250 [==============================] - 2s 677us/sample - loss: 1.0203 - accuracy: 0.4831 - val_loss: 1.0387 - val_accuracy: 0.4200
Epoch 7/80
2250/2250 [==============================] - 1s 666us/

Epoch 56/80
2250/2250 [==============================] - 2s 815us/sample - loss: 0.8021 - accuracy: 0.5800 - val_loss: 0.8579 - val_accuracy: 0.5693
Epoch 57/80
2250/2250 [==============================] - 2s 841us/sample - loss: 0.7924 - accuracy: 0.5964 - val_loss: 0.8463 - val_accuracy: 0.5987
Epoch 58/80
2250/2250 [==============================] - 2s 887us/sample - loss: 0.8014 - accuracy: 0.5876 - val_loss: 0.8285 - val_accuracy: 0.5707
Epoch 59/80
2250/2250 [==============================] - 2s 997us/sample - loss: 0.7972 - accuracy: 0.5911 - val_loss: 0.8340 - val_accuracy: 0.5640
Epoch 60/80
2250/2250 [==============================] - 2s 979us/sample - loss: 0.7951 - accuracy: 0.5942 - val_loss: 0.8204 - val_accuracy: 0.5947
Epoch 61/80
2250/2250 [==============================] - 2s 673us/sample - loss: 0.7908 - accuracy: 0.5964 - val_loss: 0.8397 - val_accuracy: 0.5707
Epoch 62/80
2250/2250 [==============================] - 2s 764us/sample - loss: 0.7931 - accuracy: 0.5991

# Evaluate your Keras model
<img src="https://www.pyimagesearch.com/wp-content/uploads/2018/09/keras_tutorial_step7.png">

In [9]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(plot)

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.56      0.64      0.60       236
        dogs       0.50      0.41      0.45       236
       panda       0.75      0.77      0.76       278

    accuracy                           0.62       750
   macro avg       0.60      0.61      0.60       750
weighted avg       0.61      0.62      0.61       750



In [10]:
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save(modelPath, save_format="h5")
f = open(label_bin, "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...
